In [14]:
import os
import numpy as np
import pandas as pd
import open3d as o3d
import cv2
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import subprocess
import tempfile
import json
import matplotlib.pyplot as plt

In [15]:
PLY_DIR = "data/andrew9/ply"
REFERENCE_FRAME = f"{PLY_DIR}/frame0000.ply"   # Reference for PSNR/SSIM
OUTPUT_CSV = "qoe_results.csv"
POINT_BUDGETS = [5000, 10000, 20000, 40000, 80000]
VIEWS = [
    ([0, 0, 0], [0, 0, 2], [0, 1, 0]),   # Front
    ([0, 0, 0], [2, 0, 0], [0, 1, 0]),   # Right
    ([0, 0, 0], [0, 2, 0], [0, 0, 1]),   # Top
]
IMG_W, IMG_H = 640, 480

## QoE model (temp)

In [16]:
def qoe_model(features):
    """
    Example QoE model.
    Replace this with your trained ML model (e.g., XGBoost, RandomForest).
    """
    # Placeholder: weighted sum
    return (
        0.5 * features["ssim"] +
        0.3 * (features["psnr"] / 50) +
        0.2 * (features["vmaf"] / 100)
    )

## Rendering functions

In [17]:
def render_view(pcd, eye, center, up, w=IMG_W, h=IMG_H):
    r = o3d.visualization.rendering.OffscreenRenderer(w, h)
    mat = o3d.visualization.rendering.MaterialRecord()
    mat.shader = "defaultUnlit"
    mat.point_size = 2

    r.scene.add_geometry("pcd", pcd, mat)
    r.scene.camera.look_at(center, eye, up)

    img = np.asarray(r.render_to_image())[:, :, :3]
    r.scene.clear_geometry()
    return img

def render_all_views(pcd):
    return [render_view(pcd, eye, center, up) for center, eye, up in VIEWS]

## Quality metrics

In [18]:
def rgb_to_y(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)[:, :, 0]

def compute_ssim_psnr(ref_imgs, dist_imgs):
    ssim_vals, psnr_vals = [], []

    for rimg, dimg in zip(ref_imgs, dist_imgs):
        y_ref, y_dist = rgb_to_y(rimg), rgb_to_y(dimg)
        ssim_vals.append(ssim(y_ref, y_dist, data_range=255))
        psnr_vals.append(psnr(y_ref, y_dist, data_range=255))

    return float(np.mean(ssim_vals)), float(np.mean(psnr_vals))


# ---------------------------------------------------------
# VMAF COMPUTATION (per frame, per-view)
# ---------------------------------------------------------

def compute_vmaf(ref_img, dist_img):
    with tempfile.TemporaryDirectory() as tmp:
        ref_path = os.path.join(tmp, "ref.png")
        dist_path = os.path.join(tmp, "dist.png")
        json_path = os.path.join(tmp, "vmaf.json")

        cv2.imwrite(ref_path, cv2.cvtColor(ref_img, cv2.COLOR_RGB2BGR))
        cv2.imwrite(dist_path, cv2.cvtColor(dist_img, cv2.COLOR_RGB2BGR))

        cmd = [
            "ffmpeg", "-y",
            "-i", ref_path,
            "-i", dist_path,
            "-lavfi",
            f"[0:v]scale=640:480,format=yuv420p[ref];"
            f"[1:v]scale=640:480,format=yuv420p[dist];"
            f"[ref][dist]libvmaf=log_path={json_path}:log_fmt=json",
            "-f", "null", "-"
        ]

        subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        with open(json_path, "r") as f:
            data = json.load(f)
            return float(data["frames"][0]["metrics"]["vmaf"])


def compute_vmaf_multi_view(ref_imgs, dist_imgs):
    vmafs = [compute_vmaf(r, d) for r, d in zip(ref_imgs, dist_imgs)]
    return float(np.mean(vmafs))

## Subsampling

In [19]:
def subsample_points(pcd, target_points):
    pts = np.asarray(pcd.points)
    N = len(pts)
    if target_points >= N:
        return pcd

    # ratio for random sampling
    ratio = target_points / N
    return pcd.random_down_sample(ratio)


In [ ]:
frame_files = sorted(
    [os.path.join(PLY_DIR, f) for f in os.listdir(PLY_DIR) if f.endswith(".ply")]
)


print(f"Found {len(frame_files)} frames.")

# Load reference point cloud
ref_pcd = o3d.io.read_point_cloud(REFERENCE_FRAME)
ref_views = render_all_views(ref_pcd)

results = []

for idx, ply_path in enumerate(frame_files):
    print(f"Processing frame {idx+1}/{len(frame_files)}: {ply_path}")

    pcd = o3d.io.read_point_cloud(ply_path)

    for T in POINT_BUDGETS:
            print(f"  Subsampling to {T} points...")

            pcd_sub = subsample_points(pcd, T)
            dist_views = render_all_views(pcd_sub)

            # Compute metrics
            ssim_val, psnr_val = compute_ssim_psnr(ref_views, dist_views)
            # vmaf_val = compute_vmaf_multi_view(ref_views, dist_views)
            vmaf_val = 0.8

            # QoE prediction
            features = {
                "ssim": ssim_val,
                "psnr": psnr_val,
                "vmaf": vmaf_val,
            }
            qoe = qoe_model(features)

            results.append({
                "frame": os.path.basename(ply_path),
                "target_points": T,
                "actual_points": len(pcd_sub.points),
                "ssim": ssim_val,
                "psnr": psnr_val,
                "vmaf": vmaf_val,
                "qoe_predicted": qoe
            })

# Save CSV
df = pd.DataFrame(results)
df.to_csv(OUTPUT_CSV, index=False)

print(f"\nDone! Saved results to {OUTPUT_CSV}")

Found 318 frames.
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
Processing frame 1/318: data/andrew9/ply/frame0000.ply
  Subsampling to 5000 points...
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
  Subsampling to 10000 points...
[Open3D INFO] EGL headless mode enabled.


/home/alan/miniconda3/envs/qoe_predict/lib/python3.10/site-packages/skimage/metrics/simple_metrics.py:168: RuntimeWarning: divide by zero encountered in scalar divide
  return 10 * np.log10((data_range**2) / err)


FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
  Subsampling to 20000 points...
[Open3D INFO] EGL headless mode enabled.
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
  Subsampling to 40000 points...
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x10338f50 (threading is enabl